In [1]:
import io
import os
import time
from datetime import datetime
from typing import Iterable

from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

In [2]:
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

In [3]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

In [4]:
assistant = client.beta.assistants.create(
    name="Opendata Bridge",
    instructions="You are a specialist in extracting tables from various files such as PDF/Excel/csv and converting them into csv files.",
    tools=[{"type": "code_interpreter"}],
    model=api_deployment_name,
)

thread = client.beta.threads.create()

In [5]:
def format_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, MessageContentText):
                print(f"{message.role}:\n{item.text.value}\n")

                if (len(item.text.annotations) > 0) and (item.text.annotations[0].file_path.file_id is not None):
                    # Retrieve file from file id
                    response_content = client.files.content(item.text.annotations[0].file_path.file_id)
                    data_in_bytes = response_content.read()
                    # Convert bytes to file
                    with open("output", "wb") as file:
                        file.write(data_in_bytes)

            elif isinstance(item, MessageContentImageFile):
                # Retrieve image from file id
                response_content = client.files.content(item.image_file.file_id)
                data_in_bytes = response_content.read()
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

In [6]:
def process_message(content: str, filename: str) -> None:

    file = None

    if filename:
        file = client.files.create(
            file=open(filename, "rb"),
            purpose='assistants'
        )

    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content, file_ids=[file.id] if file else None)
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. "
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )
    print("processing ...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            # Handle completed
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{content}\nAssistant:\n{answer}\n")
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            print(run)
            break
        if run.status == "cancelled":
            # Handle cancelled
            print(run)
            break
        if run.status == "requires_action":
            # Handle function calling and continue processing
            pass
        else:
            time.sleep(5)

In [9]:
filename = "tokyo_evacuation_site.csv"
process_message("""入力したcsvファイルの内容を読み取ってください。
                読み取ったあとは「日付」という列を追加してください。
                日付には本日の日付を値にしてください。
                最後に編集後のデータをcsvファイルで出力してください。
                """, filename)

processing ...
user:
入力したcsvファイルの内容を読み取ってください。
                読み取ったあとは「日付」という列を追加してください。
                日付には本日の日付を値にしてください。
                最後に編集後のデータをcsvファイルで出力してください。
                

assistant:
Jane Doe様、

ご指示通りにcsvファイルを編集しました。新たに「日付」列を追加し、今日の日付をこの列に設定いたしました。編集後のcsvファイルは以下のリンクからダウンロードいただけます。

[編集後のcsvファイル](sandbox:/mnt/data/edited_file.csv)

上記のリンクがご利用いただけることを確認し、何か他にご質問がありましたら、お知らせください。


In [10]:
client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)

ThreadDeleted(id='thread_4lCvd5IKemrhB4IBJXMCfu6Q', deleted=True, object='thread.deleted')